In [6]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=3, suppress=True)
sns.set_style('ticks', {'axes.grid': True, 'grid.color': '.7', 'grid.linestyle': u'--'})
sns.set_palette('colorblind')
sns.set_context('talk')